# Exemple de Stratégie d'Investissement Factorielle

Ce notebook illustre l'utilisation de la stratégie d'investissement factorielle implémentée dans ce projet. Il montre comment:

1. Récupérer et préparer les données
2. Calculer les scores factoriels
3. Construire un portefeuille basé sur ces facteurs
4. Backtester la stratégie
5. Visualiser et analyser les résultats

## Configuration et Importation des Modules

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Ajouter le répertoire parent au chemin pour pouvoir importer les modules du projet
sys.path.append('..')

from src.data.data_loader import DataLoader
from src.factors.factor_model import FactorModel
from src.factors.value_factors import ValueFactors
from src.factors.momentum_factors import MomentumFactors
from src.factors.quality_factors import QualityFactors
from src.factors.low_volatility_factors import LowVolatilityFactors
from src.portfolio.portfolio_builder import PortfolioBuilder
from src.backtest.backtest_engine import BacktestEngine
from src.utils.visualization import VisualizationTools

# Configuration du style matplotlib
VisualizationTools.set_plotting_style()

# Ignorer les avertissements
import warnings
warnings.filterwarnings('ignore')

## 1. Récupération et Préparation des Données

In [ ]:
# Configuration des paramètres
start_date = (datetime.now() - timedelta(days=365*3)).strftime('%Y-%m-%d')  # 3 ans de données
end_date = datetime.now().strftime('%Y-%m-%d')

# Liste restreinte de tickers pour l'exemple (actions du S&P 500)
tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 
    'TSLA', 'JPM', 'JNJ', 'V', 'PG', 
    'HD', 'BAC', 'MA', 'XOM', 'DIS'
]

print(f"Période d'analyse: {start_date} à {end_date}")
print(f"Nombre de tickers: {len(tickers)}")

In [ ]:
# Créer le dossier data s'il n'existe pas
if not os.path.exists('../data'):
    os.makedirs('../data')

# Initialiser le data loader
data_loader = DataLoader(start_date=start_date, end_date=end_date, data_dir='../data')

# Récupérer les données de prix (attention: peut prendre du temps)
print("Récupération des données de prix...")
price_data = data_loader.get_stock_data(tickers, save=True)

# Récupérer les données de l'indice
print("Récupération des données de l'indice S&P 500...")
market_data = data_loader.get_market_index(save=True)

# Combiner les prix de clôture ajustés en un seul DataFrame
prices_df = data_loader.combine_prices(price_data)

# Afficher un aperçu des données
print("\nAperçu des données de prix:")
prices_df.head()

In [ ]:
# Récupérer les données fondamentales (attention: peut prendre du temps)
print("Récupération des données fondamentales...")
fundamentals = data_loader.get_fundamentals(tickers, save=True)

# Afficher un exemple des données fondamentales pour un ticker
ticker_example = tickers[0]
print(f"\nExemple de données fondamentales pour {ticker_example}:")
for key in fundamentals[ticker_example].keys():
    if key == 'info':
        print(f"\n{key.capitalize()} - Quelques métriques clés:")
        info = fundamentals[ticker_example][key]
        metrics = ['sector', 'industry', 'marketCap', 'trailingPE', 'priceToBook', 'profitMargins', 'returnOnEquity']
        for metric in metrics:
            if metric in info:
                print(f"  {metric}: {info[metric]}")
    else:
        print(f"\n{key.capitalize()} - Aperçu:")
        display(fundamentals[ticker_example][key].head(3))

## 2. Calcul des Scores Factoriels

In [ ]:
# Initialiser le modèle factoriel
factor_model = FactorModel()

# Calculer les différents scores factoriels
print("Calcul des scores factoriels...")
market_series = market_data['Adj Close']

# Calculer chaque score factoriel
value_score = factor_model.calculate_value_score(prices_df, fundamentals)
momentum_score = factor_model.calculate_momentum_score(prices_df)
quality_score = factor_model.calculate_quality_score(fundamentals)
low_vol_score = factor_model.calculate_low_volatility_score(prices_df, market_series)

# Calculer le score combiné
combined_score = factor_model.calculate_combined_score(
    prices_df, fundamentals, market_series
)

# Afficher les scores de la dernière date disponible
latest_date = combined_score.index[-1]
print(f"\nScores à la date du {latest_date.strftime('%Y-%m-%d')}:")
display(combined_score.iloc[-1].sort_values(ascending=False))

In [ ]:
# Visualiser les scores des différents facteurs
factors = {
    'Value': value_score,
    'Momentum': momentum_score,
    'Quality': quality_score,
    'Low Volatility': low_vol_score,
    'Combined': combined_score
}

# Créer un DataFrame pour la dernière date
factor_scores_df = pd.DataFrame(index=tickers)

for factor_name, score_df in factors.items():
    if not score_df.empty and latest_date in score_df.index:
        factor_scores_df[factor_name] = score_df.loc[latest_date]

# Normaliser les scores pour la visualisation
normalized_scores = factor_scores_df.copy()
for col in normalized_scores.columns:
    normalized_scores[col] = (normalized_scores[col] - normalized_scores[col].mean()) / normalized_scores[col].std()

# Visualiser les scores sous forme de heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(normalized_scores, cmap='RdBu_r', annot=True, fmt='.2f', linewidths=0.5, center=0)
plt.title('Scores Factoriels Normalisés par Ticker', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 3. Construction du Portefeuille

In [ ]:
# Classer les actifs et sélectionner les meilleurs
ranks = factor_model.rank_assets(combined_score)
top_n = 5  # Sélectionner les 5 meilleurs pour l'exemple
selected_assets = factor_model.select_top_assets(ranks, top_n=top_n)

# Afficher les actifs sélectionnés à la dernière date
print(f"Actifs sélectionnés au {latest_date.strftime('%Y-%m-%d')}:")
selected_tickers = selected_assets.iloc[-1][selected_assets.iloc[-1]].index.tolist()
for ticker in selected_tickers:
    print(f"- {ticker}: score = {combined_score.iloc[-1][ticker]:.4f}")

In [ ]:
# Initialiser le constructeur de portefeuille
portfolio_builder = PortfolioBuilder()

# Calculer les rendements pour l'optimisation
returns_df = prices_df.pct_change().dropna()

# Comparer différentes méthodes de construction de portefeuille
portfolio_methods = {
    'Equal Weight': portfolio_builder.equal_weight(selected_assets),
    'Score Weighted': portfolio_builder.score_weighted(selected_assets, combined_score),
    'Minimum Variance': portfolio_builder.minimum_variance(selected_assets, returns_df),
    'Maximum Sharpe': portfolio_builder.maximum_sharpe_ratio(selected_assets, returns_df),
    'Risk Parity': portfolio_builder.risk_parity(selected_assets, returns_df)
}

# Visualiser les poids pour chaque méthode à la dernière date
weights_df = pd.DataFrame(index=tickers)

for method_name, weights in portfolio_methods.items():
    if not weights.empty and latest_date in weights.index:
        weights_df[method_name] = weights.loc[latest_date]

# Visualiser les poids
ax = weights_df.plot(kind='bar', figsize=(14, 6))
plt.title('Poids du Portefeuille par Méthode de Construction', fontsize=16)
plt.xlabel('Ticker', fontsize=12)
plt.ylabel('Poids', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Méthode')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 4. Backtest de la Stratégie

In [ ]:
# Initialiser le moteur de backtest
backtest_engine = BacktestEngine(initial_capital=10000, transaction_costs=0.001)

# Exécuter les backtests pour chaque méthode de construction
backtest_results = {}

print("Exécution des backtests...")
for name, weights in portfolio_methods.items():
    print(f"Backtest de la stratégie: {name}")
    equity_curve, metrics, positions, trades = backtest_engine.run_backtest(weights, prices_df)
    backtest_results[name] = (equity_curve, metrics, positions, trades)
    
    # Afficher quelques métriques clés
    print(f"  - Rendement annualisé: {metrics['annual_return']:.2%}")
    print(f"  - Volatilité annualisée: {metrics['volatility']:.2%}")
    print(f"  - Ratio de Sharpe: {metrics['sharpe_ratio']:.2f}")
    print(f"  - Drawdown maximal: {metrics['max_drawdown']:.2%}\n")

In [ ]:
# Comparer les performances des différentes stratégies
comparison = backtest_engine.compare_strategies(backtest_results)

# Formater la table pour l'affichage
display_table = comparison.copy()
percentage_cols = ['total_return', 'annual_return', 'volatility', 'max_drawdown', 'win_rate']

for col in display_table.columns:
    if col in percentage_cols:
        display_table[col] = display_table[col].apply(lambda x: f"{x:.2%}")
    else:
        display_table[col] = display_table[col].apply(lambda x: f"{x:.2f}")

display(display_table)

## 5. Visualisation et Analyse des Résultats

In [ ]:
# Tracer les courbes d'équité
equity_curves = {name: results[0] for name, results in backtest_results.items()}

fig, ax = plt.subplots(figsize=(14, 8))

for name, curve in equity_curves.items():
    normalized_curve = curve / curve.iloc[0] * 100
    normalized_curve.plot(ax=ax, label=name, linewidth=2)

# Ajouter l'indice de référence
benchmark_curve = market_series.loc[equity_curves['Equal Weight'].index[0]:equity_curves['Equal Weight'].index[-1]]
normalized_benchmark = benchmark_curve / benchmark_curve.iloc[0] * 100
normalized_benchmark.plot(ax=ax, label='S&P 500', linestyle='--', color='black', linewidth=2)

ax.set_title('Performance des Stratégies Factorielles', fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Performance (%)', fontsize=14)
ax.legend(title='Stratégie')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Trouver la meilleure stratégie basée sur le rendement annualisé
best_strategy = comparison['annual_return'].idxmax()
print(f"Meilleure stratégie: {best_strategy}")

# Tracer les drawdowns pour la meilleure stratégie
fig = VisualizationTools.plot_drawdowns(equity_curves[best_strategy])
plt.show()

# Tracer les rendements mensuels pour la meilleure stratégie
fig = VisualizationTools.plot_monthly_returns_heatmap(equity_curves[best_strategy])
plt.show()

In [ ]:
# Analyser la composition du portefeuille au fil du temps
best_weights = portfolio_methods[best_strategy]

# Afficher l'évolution des poids pour les principaux actifs
top_assets = best_weights.sum().sort_values(ascending=False).head(5).index
weights_evolution = best_weights[top_assets]

# Rééchantillonner pour une visualisation plus claire (mensuelle)
monthly_weights = weights_evolution.resample('M').last()

# Tracer l'évolution des poids
plt.figure(figsize=(14, 8))
monthly_weights.plot.area(alpha=0.7)
plt.title(f'Évolution des Poids du Portefeuille - {best_strategy}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Allocation (%)', fontsize=14)
plt.legend(title='Ticker')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Analyser les métriques glissantes pour la meilleure stratégie
fig = VisualizationTools.plot_rolling_metrics(
    equity_curves[best_strategy], 
    window=63,  # ~3 mois
    metrics=['return', 'volatility', 'sharpe']
)
plt.show()

In [ ]:
# Examiner la contribution des facteurs (visualisation simplifiée)
factor_scores_dict = {
    'Value': value_score,
    'Momentum': momentum_score,
    'Quality': quality_score,
    'Low Volatility': low_vol_score
}

# Tracer l'exposition aux facteurs à la date la plus récente
fig = VisualizationTools.plot_factor_exposure(
    best_weights,
    factor_scores_dict,
    date=latest_date
)
plt.title(f'Exposition aux Facteurs - {best_strategy} ({latest_date.strftime("%Y-%m-%d")})', fontsize=16)
plt.show()

In [ ]:
# Afficher les transactions pour la meilleure stratégie
trades_df = backtest_results[best_strategy][3]

if not trades_df.empty:
    # Calculer quelques statistiques sur les trades
    num_trades = len(trades_df)
    avg_trade_size = trades_df['value'].abs().mean()
    total_costs = trades_df['transaction_cost'].sum()
    
    print(f"Analyse des transactions pour la stratégie {best_strategy}:")
    print(f"Nombre total de transactions: {num_trades}")
    print(f"Taille moyenne des transactions: ${avg_trade_size:.2f}")
    print(f"Coûts de transaction totaux: ${total_costs:.2f}")
    
    # Afficher les 10 dernières transactions
    print("\nDernières transactions:")
    display(trades_df.tail(10))
else:
    print("Aucune transaction enregistrée pour cette stratégie.")

## 6. Conclusion et Prochaines Étapes

Dans ce notebook, nous avons implémenté et testé une stratégie d'investissement factorielle complète. Les principales étapes ont été :

1. **Récupération et préparation des données** : Prix historiques et données fondamentales
2. **Calcul des scores factoriels** : Value, momentum, qualité et faible volatilité
3. **Construction du portefeuille** : Différentes méthodes d'allocation d'actifs
4. **Backtest de la stratégie** : Évaluation des performances historiques
5. **Analyse des résultats** : Visualisations et métriques

Cette implémentation démontre comment combiner des facteurs bien connus pour construire une stratégie d'investissement systématique. Les résultats montrent que la stratégie factorielle peut potentiellement surperformer le marché, surtout avec des méthodes d'allocation plus sophistiquées.

### Prochaines étapes possibles :

- **Optimisation des paramètres** : Ajuster les poids des facteurs et les seuils de sélection
- **Ajout de nouveaux facteurs** : Explorer d'autres facteurs comme le sentiment ou les facteurs macroéconomiques
- **Tests de robustesse** : Analyse de sensibilité et tests sur différentes périodes de marché
- **Implémentation en temps réel** : Automatisation de la collecte de données et des signaux de trading
- **Diversification** : Application à d'autres classes d'actifs ou régions géographiques